<a href="https://colab.research.google.com/github/rajesh-bhat/spark-ai-summit-2020-text-extraction/blob/master/CRNN_CTC_Wandb_sweeps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade wandb

     |████████████████████████████████| 1.4MB 7.3MB/s 
     |████████████████████████████████| 163kB 38.6MB/s 
     |████████████████████████████████| 102kB 11.0MB/s 
     |████████████████████████████████| 112kB 252kB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 71kB 6.0MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=85b71e610de3f2ff02f07f150a3f6241afb5e0c389293b9c7aad19ee65a3ff9e
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=2cabc98b6a307f60111a5ca48d3019836b218eeb153499e0ed80f19cbbe7329a
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73870 sha256=4d7f

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


import cv2
import numpy as np
import string
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional

from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *

from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tqdm import tqdm
from collections import Counter
from PIL import Image


import wandb
from wandb.keras import WandbCallback

In [ ]:
tf.__version__

'2.3.0'

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip /content/drive/My\ Drive/mjsynth_sample.zip

Streaming output truncated to the last 5000 lines.
  inflating: mjsynth_sample/94_Undisclosed_82317.jpg  
  inflating: mjsynth_sample/93_Affiliating_1411.jpg  
  inflating: mjsynth_sample/92_COEQUALS_14685.jpg  
  inflating: mjsynth_sample/91_Shadowier_69823.jpg  
  inflating: mjsynth_sample/90_acquired_811.jpg  
  inflating: mjsynth_sample/89_snoopier_72273.jpg  
  inflating: mjsynth_sample/88_HUGHES_37216.jpg  
  inflating: mjsynth_sample/87_Depopulates_20746.jpg  
  inflating: mjsynth_sample/86_REVIEWS_65747.jpg  
  inflating: mjsynth_sample/85_Craniums_17804.jpg  
  inflating: mjsynth_sample/84_MISCONCEIVED_48832.jpg  
  inflating: mjsynth_sample/83_philately_57195.jpg  
  inflating: mjsynth_sample/82_Bullfrogs_10133.jpg  
  inflating: mjsynth_sample/81_dengue_20578.jpg  
  inflating: mjsynth_sample/80_preregistering_59794.jpg  
  inflating: mjsynth_sample/79_FOUNDER_30506.jpg  
  inflating: mjsynth_sample/78_millimeters_48468.jpg  
  inflating: mjsynth_sample/77_endear_25654.jpg  

In [ ]:
final_paths = []
final_texts = []

data_folder = "mjsynth_sample"
for path in os.listdir(data_folder):

    final_paths.append(data_folder + "/" + path)
    final_texts.append(path.split("_")[1])

In [ ]:
final_paths[:10], final_texts[:10]

(['mjsynth_sample/188_Heady_35420.jpg',
  'mjsynth_sample/365_minuter_48713.jpg',
  'mjsynth_sample/202_okras_53257.jpg',
  'mjsynth_sample/217_taliban_77309.jpg',
  'mjsynth_sample/30_ABROGATING_386.jpg',
  'mjsynth_sample/124_Hawker_35246.jpg',
  'mjsynth_sample/22_truffle_80940.jpg',
  'mjsynth_sample/297_Toolmaker_79626.jpg',
  'mjsynth_sample/11_FOOTAGE_30043.jpg',
  'mjsynth_sample/95_diamondbacks_21420.jpg'],
 ['Heady',
  'minuter',
  'okras',
  'taliban',
  'ABROGATING',
  'Hawker',
  'truffle',
  'Toolmaker',
  'FOOTAGE',
  'diamondbacks'])

In [ ]:
### get vocab from dataset
vocab = set("".join(map(str, final_texts)))
print(sorted(vocab))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
Counter("".join(map(str, final_texts)))

Counter({'0': 38,
         '1': 56,
         '2': 30,
         '3': 21,
         '4': 26,
         '5': 16,
         '6': 16,
         '7': 15,
         '8': 17,
         '9': 10,
         'A': 12188,
         'B': 3874,
         'C': 7202,
         'D': 6379,
         'E': 16729,
         'F': 2799,
         'G': 4768,
         'H': 3898,
         'I': 12696,
         'J': 518,
         'K': 1611,
         'L': 7997,
         'M': 4745,
         'N': 10222,
         'O': 8783,
         'P': 5178,
         'Q': 350,
         'R': 11125,
         'S': 14094,
         'T': 10070,
         'U': 4990,
         'V': 1787,
         'W': 1806,
         'X': 428,
         'Y': 2289,
         'Z': 696,
         'a': 21612,
         'b': 4718,
         'c': 10014,
         'd': 9727,
         'e': 31936,
         'f': 3274,
         'g': 8370,
         'h': 6096,
         'i': 24247,
         'j': 453,
         'k': 2761,
         'l': 14693,
         'm': 6928,
         'n': 20242,
         'o'

In [ ]:
char_list = sorted(vocab)

def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print(char)
        
    return dig_lst

In [ ]:
encode_to_labels("HELLO")

[17, 14, 21, 21, 24]

In [ ]:
train_final_paths = final_paths[ : int(len(final_paths) * 0.90)]
train_final_texts = final_texts[ : int(len(final_texts) * 0.90)]

val_final_paths = final_paths[int(len(final_paths) * 0.90) : ]
val_final_texts = final_texts[int(len(final_texts) * 0.90) : ]

In [ ]:
len(train_final_paths), len(val_final_paths)

(47427, 5270)

In [ ]:
max_label_len = max([len(str(text)) for text in final_texts])

In [ ]:
max_label_len

23

In [ ]:
class My_Generator(Sequence):
    
    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        
        batch_paths = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_texts = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        images = []
        training_txt = []
        train_label_length = []
        train_input_length = []

        for im_path, text in zip(batch_paths, batch_texts):
            
            try:
                text = str(text).strip()
                img = cv2.cvtColor(cv2.imread(im_path), cv2.COLOR_BGR2GRAY)   

                ### actually returns h, w
                h, w = img.shape

                ### if height less than 32
                if h < 32:
                    add_zeros = np.ones((32-h, w)) * 255
                    img = np.concatenate((img, add_zeros))
                    h = 32

                ## if width less than 128
                if w < 128:
                    add_zeros = np.ones((h, 128-w)) * 255
                    img = np.concatenate((img, add_zeros), axis=1)
                    w = 128

                ### if width is greater than 128 or height greater than 32
                if w > 128 or h > 32:
                    img = cv2.resize(img, (128, 32))

                img = np.expand_dims(img , axis = 2)

                # Normalize each image
                img = img / 255.

                images.append(img)
                training_txt.append(encode_to_labels(text))
                train_label_length.append(len(text))
                train_input_length.append(31)
            except:
                
                pass

        return [np.array(images), 
               pad_sequences(training_txt, maxlen=max_label_len, padding='post', value=len(char_list)), 
               np.array(train_input_length), 
               np.array(train_label_length)], np.zeros(len(images))

In [ ]:
train_final_paths[:1], train_final_texts[:1]

(['mjsynth_sample/188_Heady_35420.jpg'], ['Heady'])

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'learning_rate': {
            'values': [1e-2, 1e-3, 3e-4]
        },
        'optimizer': {
            'values': ['adam', 'nadam', 'sgd']
        },
        'activation': {
            'values': ['relu', 'elu', 'selu']
        }
    }
}

In [ ]:
config_defaults = {
    'epochs': 30,
    'batch_size': 256,
    'weight_decay': 0.0005,
    'learning_rate': 1e-3,
    'activation': 'relu',
    'optimizer': 'nadam',
    'dropout': 0.5,
    'momentum': 0.9,
    'seed': 42
}


In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="authors", project="text-recognition-crnn-ctc")

Create sweep with ID: lqf5dhae
Sweep URL: https://app.wandb.ai/authors/text-recognition-crnn-ctc/sweeps/lqf5dhae


In [ ]:
train_generator = My_Generator(train_final_paths, train_final_texts, config_defaults["batch_size"])
val_generator = My_Generator(val_final_paths, val_final_texts, config_defaults["batch_size"])

In [ ]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args 
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)


def train():

    # Initialize a new wandb run
    wandb.init(config=config_defaults)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    # input with shape of height=32 and width=128 
    inputs = Input(shape=(32, 128, 1))

    conv_1 = Conv2D(32, (3,3), activation = config.activation, padding='same')(inputs)
    pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
    
    conv_2 = Conv2D(64, (3,3), activation = config.activation, padding='same')(pool_1)
    pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)

    conv_3 = Conv2D(128, (3,3), activation = config.activation, padding='same')(pool_2)
    conv_4 = Conv2D(128, (3,3), activation = config.activation, padding='same')(conv_3)
    pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
    
    conv_5 = Conv2D(256, (3,3), activation = config.activation, padding='same')(pool_4)
    # Batch normalization layer
    batch_norm_5 = BatchNormalization()(conv_5)
    
    conv_6 = Conv2D(256, (3,3), activation = config.activation, padding='same')(batch_norm_5)
    batch_norm_6 = BatchNormalization()(conv_6)
    pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
    
    conv_7 = Conv2D(64, (2,2), activation = config.activation)(pool_6)
    
    squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)
    
    # bidirectional LSTM layers with units=128
    blstm_1 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(squeezed)
    blstm_2 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(blstm_1)

    outputs = Dense(len(char_list) + 1, activation = 'softmax')(blstm_2)

    # model to be used at test time
    act_model = Model(inputs, outputs)


    # Define the optimizer
    if config.optimizer=='sgd':
        optimizer = SGD(lr=config.learning_rate, decay=1e-5, momentum=config.momentum, nesterov=True)
    elif config.optimizer=='adam':
        optimizer = Adam(lr=config.learning_rate, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
    elif config.optimizer=='nadam':
        optimizer = Nadam(lr=config.learning_rate, beta_1=0.9, beta_2=0.999, clipnorm=1.0)


    labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')

    
    loss_out = Lambda(ctc_lambda_func, 
                    output_shape=(1,), 
                    name='ctc')([outputs, labels, input_length, label_length])

    #model to be used at training time
    model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)


    file_path = "C_LSTM_best.hdf5"

    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = optimizer)

    checkpoint = ModelCheckpoint(filepath=file_path, 
                                monitor='val_loss', 
                                verbose=1, 
                                save_best_only=True, 
                                mode='min')

    callbacks_list = [checkpoint, 
                      WandbCallback(monitor="val_loss", mode="min"), 
                      EarlyStopping(patience=5, restore_best_weights=True)]

    history = model.fit(train_generator, 
                        epochs = 30,
                        steps_per_epoch = len(train_final_paths) // config.batch_size,
                        validation_data=val_generator,
                        validation_steps = len(val_final_paths) // config.batch_size,
                        verbose = 1,
                        callbacks = callbacks_list,
                        shuffle=True)

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: shwtble3 with config:
	activation: elu
	learning_rate: 0.0003
	optimizer: nadam
wandb: Agent Started Run: shwtble3


Epoch 1/30
185/185 [==============================] - ETA: 0s - loss: 31.6124
Epoch 00001: val_loss improved from inf to 39.82682, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 79s 424ms/step - loss: 31.6124 - val_loss: 39.8268
Epoch 2/30
185/185 [==============================] - ETA: 0s - loss: 27.2007
Epoch 00002: val_loss improved from 39.82682 to 28.64357, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 76s 410ms/step - loss: 27.2007 - val_loss: 28.6436
Epoch 3/30
185/185 [==============================] - ETA: 0s - loss: 24.2040
Epoch 00003: val_loss improved from 28.64357 to 23.55156, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 77s 415ms/step - loss: 24.2040 - val_loss: 23.5516
Epoch 4/30
185/185 [==============================] - ETA: 0s - loss: 19.3888
Epoch 00004: val_loss improved from 23.55156 to 16.96192, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 77s 

Epoch 1/30
185/185 [==============================] - ETA: 0s - loss: 30.8190
Epoch 00001: val_loss improved from inf to 32.56268, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 79s 427ms/step - loss: 30.8190 - val_loss: 32.5627
Epoch 2/30
185/185 [==============================] - ETA: 0s - loss: 25.6927
Epoch 00002: val_loss improved from 32.56268 to 28.30522, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 75s 407ms/step - loss: 25.6927 - val_loss: 28.3052
Epoch 3/30
185/185 [==============================] - ETA: 0s - loss: 18.5138
Epoch 00003: val_loss improved from 28.30522 to 19.41685, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 75s 407ms/step - loss: 18.5138 - val_loss: 19.4168
Epoch 4/30
185/185 [==============================] - ETA: 0s - loss: 9.0676
Epoch 00004: val_loss improved from 19.41685 to 8.38917, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 75s 40

Epoch 1/30
185/185 [==============================] - ETA: 0s - loss: 32.4350
Epoch 00001: val_loss improved from inf to 30.97518, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 76s 409ms/step - loss: 32.4350 - val_loss: 30.9752
Epoch 2/30
185/185 [==============================] - ETA: 0s - loss: 30.6266
Epoch 00002: val_loss improved from 30.97518 to 30.94754, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 72s 390ms/step - loss: 30.6266 - val_loss: 30.9475
Epoch 3/30
185/185 [==============================] - ETA: 0s - loss: 30.3173
Epoch 00003: val_loss did not improve from 30.94754
185/185 [==============================] - 71s 386ms/step - loss: 30.3173 - val_loss: 31.7945
Epoch 4/30
185/185 [==============================] - ETA: 0s - loss: 29.5657
Epoch 00004: val_loss improved from 30.94754 to 29.77844, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 72s 391ms/step - loss: 29.5657 - val_loss: 

Epoch 1/30
185/185 [==============================] - ETA: 0s - loss: 31.4045
Epoch 00001: val_loss improved from inf to 33.20848, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 77s 418ms/step - loss: 31.4045 - val_loss: 33.2085
Epoch 2/30
185/185 [==============================] - ETA: 0s - loss: 27.6030
Epoch 00002: val_loss improved from 33.20848 to 28.23342, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 74s 401ms/step - loss: 27.6030 - val_loss: 28.2334
Epoch 3/30
185/185 [==============================] - ETA: 0s - loss: 25.7309
Epoch 00003: val_loss improved from 28.23342 to 24.65734, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 74s 401ms/step - loss: 25.7309 - val_loss: 24.6573
Epoch 4/30
185/185 [==============================] - ETA: 0s - loss: 22.3544
Epoch 00004: val_loss improved from 24.65734 to 20.47820, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 74s 

...

Epoch 1/30
185/185 [==============================] - ETA: 0s - loss: 31.9376
Epoch 00001: val_loss improved from inf to 30.53888, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 74s 399ms/step - loss: 31.9376 - val_loss: 30.5389
Epoch 2/30
185/185 [==============================] - ETA: 0s - loss: 30.2745
Epoch 00002: val_loss improved from 30.53888 to 30.14413, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 70s 378ms/step - loss: 30.2745 - val_loss: 30.1441
Epoch 3/30
185/185 [==============================] - ETA: 0s - loss: 29.8768
Epoch 00003: val_loss improved from 30.14413 to 29.83395, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 70s 378ms/step - loss: 29.8768 - val_loss: 29.8339
Epoch 4/30
185/185 [==============================] - ETA: 0s - loss: 29.7056
Epoch 00004: val_loss improved from 29.83395 to 29.72552, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 70s 

Epoch 1/30
185/185 [==============================] - ETA: 0s - loss: 31.9493
Epoch 00001: val_loss improved from inf to 32.09550, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 75s 406ms/step - loss: 31.9493 - val_loss: 32.0955
Epoch 2/30
185/185 [==============================] - ETA: 0s - loss: 27.6222
Epoch 00002: val_loss improved from 32.09550 to 28.74405, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 73s 393ms/step - loss: 27.6222 - val_loss: 28.7441
Epoch 3/30
185/185 [==============================] - ETA: 0s - loss: 25.6939
Epoch 00003: val_loss improved from 28.74405 to 25.10946, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 72s 392ms/step - loss: 25.6939 - val_loss: 25.1095
Epoch 4/30
185/185 [==============================] - ETA: 0s - loss: 22.3230
Epoch 00004: val_loss improved from 25.10946 to 20.57547, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 72s 

Epoch 1/30
185/185 [==============================] - ETA: 0s - loss: 31.2619
Epoch 00001: val_loss improved from inf to 28.76311, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 77s 418ms/step - loss: 31.2619 - val_loss: 28.7631
Epoch 2/30
185/185 [==============================] - ETA: 0s - loss: 26.7206
Epoch 00002: val_loss improved from 28.76311 to 27.89942, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 75s 403ms/step - loss: 26.7206 - val_loss: 27.8994
Epoch 3/30
185/185 [==============================] - ETA: 0s - loss: 22.9623
Epoch 00003: val_loss improved from 27.89942 to 21.28388, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 74s 403ms/step - loss: 22.9623 - val_loss: 21.2839
Epoch 4/30
185/185 [==============================] - ETA: 0s - loss: 17.3103
Epoch 00004: val_loss improved from 21.28388 to 14.58513, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 74s 

Epoch 1/30
185/185 [==============================] - ETA: 0s - loss: 32.2076
Epoch 00001: val_loss improved from inf to 31.09466, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 78s 420ms/step - loss: 32.2076 - val_loss: 31.0947
Epoch 2/30
185/185 [==============================] - ETA: 0s - loss: 29.9204
Epoch 00002: val_loss improved from 31.09466 to 30.07341, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 74s 401ms/step - loss: 29.9204 - val_loss: 30.0734
Epoch 3/30
185/185 [==============================] - ETA: 0s - loss: 29.8057
Epoch 00003: val_loss improved from 30.07341 to 30.02385, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 75s 403ms/step - loss: 29.8057 - val_loss: 30.0238
Epoch 4/30
185/185 [==============================] - ETA: 0s - loss: 29.5737
Epoch 00004: val_loss did not improve from 30.02385
185/185 [==============================] - 74s 398ms/step - loss: 29.5737 - val_loss: 

Epoch 1/30
185/185 [==============================] - ETA: 0s - loss: 32.3242
Epoch 00001: val_loss improved from inf to 31.07723, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 74s 399ms/step - loss: 32.3242 - val_loss: 31.0772
Epoch 2/30
185/185 [==============================] - ETA: 0s - loss: 30.5546
Epoch 00002: val_loss did not improve from 31.07723
185/185 [==============================] - 71s 382ms/step - loss: 30.5546 - val_loss: 31.3554
Epoch 3/30
185/185 [==============================] - ETA: 0s - loss: 29.9379
Epoch 00003: val_loss did not improve from 31.07723
185/185 [==============================] - 70s 380ms/step - loss: 29.9379 - val_loss: 31.9193
Epoch 4/30
185/185 [==============================] - ETA: 0s - loss: 28.9863
Epoch 00004: val_loss improved from 31.07723 to 30.26583, saving model to C_LSTM_best.hdf5
185/185 [==============================] - 70s 381ms/step - loss: 28.9863 - val_loss: 30.2658
Epoch 5/30
147/185 [===========

In [ ]:
def pre_process_image(path):
    
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY)
    ### actually returns h, w
    h, w = img.shape

    ### if height less than 32
    if h < 32:
        add_zeros = np.ones((32-h, w)) * 255
        img = np.concatenate((img, add_zeros))
        h = 32

    ## if width less than 128
    if w < 128:
        add_zeros = np.ones((h, 128-w)) * 255
        img = np.concatenate((img, add_zeros), axis=1)
        w = 128

    ### if width is greater than 128 or height greater than 32
    if w > 128 or h > 32:
        img = cv2.resize(img, (128, 32))

    
    img = np.expand_dims(img , axis = 2)

    # Normalize each image
    img = img / 255.
    
    return img

In [ ]:
act_model.load_weights('C_LSTM_best.hdf5')

In [ ]:
def predict_output(img):
    # predict outputs on validation images
    prediction = act_model.predict(np.array([img]))
    ## shape (batch_size, num_timesteps, vocab_size)

    # use CTC decoder
    out = K.get_value(K.ctc_decode(prediction, 
                                   input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                                   greedy=True)[0][0])

    ## get the final text    
    for x in out:

        print("predicted text = ", end = '')

        for p in x:

            if int(p) != -1:
                print(char_list[int(p)], end = '')
        
        print('\n')

In [ ]:
from IPython.display import display
from random import randrange

for i in range(10):
    index = randrange(len(val_final_paths))
    im = Image.open(val_final_paths[index])
    display(im)

    print("\n")

    test_img = pre_process_image(val_final_paths[index])
    predict_output(test_img)




Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
predicted text = Dowel





predicted text = Choirboys





predicted text = congrats





predicted text = entlaaee





predicted text = scapegoating





predicted text = outed





predicted text = fulsomeneer





predicted text = jalapenos





predicted text = ANTIQUATING





predicted text = hitler



In [ ]:
## results can be improved with finetuning and including more data :)